In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path_name = '/kaggle/input/creditcard/creditcard.csv'

In [ ]:

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
Labels = ['fraud','valid']
%matplotlib inline

In [ ]:
sns.set_style('darkgrid')


In [ ]:
details = pd.read_csv(path_name)
details.head()

In [ ]:
details.shape
details.info()

In [ ]:
details.describe()

In [ ]:
details['Class'].describe()
details['Class'].value_counts() #here 0 is valid transactions and 1 represents fraud.

In [ ]:
sns.countplot(details.Class)
LABELS = Labels
plt.title('transaction classification')
plt.xlabel('classification')
plt.ylabel('frequency')
plt.show()

In [ ]:
details.corr()

In [ ]:
details.shape
details.describe()

In [ ]:
#plotting the correlation among data
plt.figure(figsize=(18,18))
sns.set(font_scale=0.7)
sns.heatmap(details.corr(),cmap='viridis',annot=True)
plt.show()
#shows that the correlation is decently low or enough for us to not take into account.

In [ ]:
#classifying dataset into valid and fraud.
fraud = details[details.Class == 1]

valid = details[details.Class == 0]
valid.shape, fraud.shape, (fraud.size/(fraud.size+valid.size))*100

In [ ]:
fraud.Amount.describe()

In [ ]:
valid.Amount.describe()

In [ ]:
smoler_details = details.sample(frac=0.4)
smoler_details.shape

In [ ]:
print(len(fraud)/len(valid))

In [ ]:
#creating a random state variable
state=np.random.RandomState(69)

In [ ]:
y = smoler_details['Class']
X = smoler_details.drop(['Class'],axis=1)
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0],X.shape[1]))
X.shape

In [ ]:
#Initializing and describing models
classifiers = {
    'isolationforest': IsolationForest(n_estimators=100,max_samples=len(X),random_state=state,verbose=0,contamination=(len(fraud)/len(valid))),
    'localoutlierfactor': LocalOutlierFactor(n_neighbors=20,algorithm='auto',leaf_size=30,p=2,metric='minkowski',metric_params=None,contamination=((len(fraud)/len(valid)))),
    'supportvm': OneClassSVM(kernel='rbf', degree=3, gamma=0.1,nu=0.05, max_iter=-1)
}

In [ ]:
for i,(clf_name,clf) in enumerate(classifiers.items()):
    
    if clf_name == 'isolationforest':
        clf.fit(X)
        predictions = clf.predict(X)
        prediction_score = clf.decision_function(X)
    elif clf_name == 'localoutlierfactor':
        predictions = clf.fit_predict(X)
        prediction_score = clf.negative_outlier_factor_
    else:
        clf.fit(X)
        predictions = clf.predict(X)
    
    #changing the prediction values to fit the fraud,valid labelling system
    
    predictions[predictions == 1] = 0
    predictions[predictions == -1] = 1
    n_errors = (predictions != y).sum()
    
    #printing the final results
    
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(y,predictions)*100)
    print("Classification Report :")
    print(classification_report(y,predictions))